<a href="https://colab.research.google.com/github/SanjayBhargavKudupudi/DATA_MINING/blob/main/video/video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from moviepy.editor import VideoFileClip

# Load the video file
video_path = '/content/drive/MyDrive/DATA_MINING_DATASETS/videoplayback (online-video-cutter.com).mp4'
video = VideoFileClip(video_path)

# Extract basic metadata
video_duration = video.duration
video_fps = video.fps
video_resolution = video.size

video_duration, video_fps, video_resolution


(76.6, 30.0, [640, 360])

In [ ]:
from PIL import Image
import numpy as np
import io

# Function to extract and save a frame from the video at a specific timestamp
def extract_frame(video, timestamp):
    frame = video.get_frame(timestamp)
    frame_image = Image.fromarray((frame * 255).astype(np.uint8))
    return frame_image

# Extracting frames from different parts of the video
timestamps = [0, video_duration / 2, video_duration - 1]  # Start, middle, end
extracted_frames = [extract_frame(video, t) for t in timestamps]

# Display the first extracted frame as a sample
extracted_frames[0].show()


In [ ]:
from PIL import ImageOps
import numpy as np
import cv2

def calculate_brightness(image):
    """Calculate the average brightness of the image"""
    greyscale_image = ImageOps.grayscale(image)
    histogram = greyscale_image.histogram()
    pixels = sum(histogram)
    brightness = scale = len(histogram)

    for index in range(0, scale):
        ratio = histogram[index] / pixels
        brightness += ratio * -scale + index

    return brightness / scale

def calculate_color_distribution(image):
    """Calculate the color distribution (dark vs light pixels)"""
    greyscale_image = ImageOps.grayscale(image)
    dark_pixels = np.sum(np.array(greyscale_image) < 128)
    light_pixels = np.sum(np.array(greyscale_image) >= 128)
    total_pixels = dark_pixels + light_pixels
    return dark_pixels / total_pixels, light_pixels / total_pixels

def calculate_edge_density(image):
    """Calculate the edge density using Canny edge detection"""
    image_array = np.array(image)
    edges = cv2.Canny(image_array,100,200)
    edge_density = np.sum(edges > 0) / (edges.shape[0] * edges.shape[1])
    return edge_density

# Extracting features for each frame
features = []
for frame in extracted_frames:
    brightness = calculate_brightness(frame)
    dark_ratio, light_ratio = calculate_color_distribution(frame)
    edge_density = calculate_edge_density(frame)
    features.append({
        "brightness": brightness,
        "dark_ratio": dark_ratio,
        "light_ratio": light_ratio,
        "edge_density": edge_density
    })

features


[{'brightness': 127.49999999999999,
  'dark_ratio': 0.5478038194444445,
  'light_ratio': 0.45219618055555555,
  'edge_density': 0.10598958333333333},
 {'brightness': 127.50000000000004,
  'dark_ratio': 0.7418402777777777,
  'light_ratio': 0.2581597222222222,
  'edge_density': 0.05950086805555556},
 {'brightness': 127.49999999999994,
  'dark_ratio': 0.3942100694444444,
  'light_ratio': 0.6057899305555555,
  'edge_density': 0.10239583333333334}]

In [ ]:
import cv2

cap = cv2.VideoCapture('video.mp4')

ret, frame1 = cap.read()
ret, frame2 = cap.read()

while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2)
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY)
    dilated = cv2.dilate(thresh, None, iterations=3)
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) < 900:
            continue
        # Motion detected, process accordingly

    frame1 = frame2
    ret, frame2 = cap.read()

    if not ret:
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
!pip install opencv-python



In [ ]:
from google.colab.patches import cv2_imshow
import cv2

# Initialize video source (the video file)
cap = cv2.VideoCapture('/content/drive/MyDrive/DATA_MINING_DATASETS/videoplayback (online-video-cutter.com).mp4')

# Read two frames from the video, initially
ret, frame1 = cap.read()
ret, frame2 = cap.read()

while cap.isOpened():
    # ... (rest of the code remains the same)

    # Display the resulting frame
    cv2_imshow(frame1)

    # Read the next frame
    frame1 = frame2
    ret, frame2 = cap.read()

    # Breaking the loop if 'q' is pressed - This won't work in Colab, so we use a fixed number of frames instead
    if not ret:
        break

# Release the video capture object
cap.release()


In [ ]:
import cv2
import os

# Create a directory to save the extracted frames
frames_dir = "extracted_frames"
os.makedirs(frames_dir, exist_ok=True)

# Load the video
video_path = '/content/drive/MyDrive/DATA_MINING_DATASETS/videoplayback (online-video-cutter.com).mp4'
cap = cv2.VideoCapture(video_path)

# Frame extraction settings
interval = 30  # Extract a frame every 30 seconds
frame_rate = cap.get(cv2.CAP_PROP_FPS)
count = 0

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    if count % (frame_rate * interval) == 0:
        frame_name = os.path.join(frames_dir, f"frame_{count // (frame_rate * interval)}.jpg")
        cv2.imwrite(frame_name, frame)

    count += 1

cap.release()
